In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)
options(repr.matrix.max.rows = 6)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [2]:
# loads data using read_csv function
players_origin <- read_csv("https://raw.githubusercontent.com/mcheng250/DSCI_Project_final_report/refs/heads/main/data/players.csv")
sessions_origin <- read_csv("https://raw.githubusercontent.com/mcheng250/DSCI_Project_final_report/refs/heads/main/data/sessions.csv")

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
# Clean up the sessions.csv: mutate start_time and end_time to get session_length, use dmy_hm to turn start_time and 
# end_time into proper format, use difftime to calculate session length
# wrangle the session_length so we have avg_session_length. 
# Clean other columns so all we left is hashedEmail and avg_session_length while rounding the avg_session_length 
# decimal place so the data looks more clean
sessions_tidy <- sessions_origin |>
                    mutate(start_time = dmy_hm(start_time),
                           end_time = dmy_hm(end_time),
                           session_length = as.numeric(difftime(end_time, start_time, units = "mins"))) |>
                    group_by(hashedEmail) |>
                    summarize(avg_session_length = round(mean(session_length, na.rm = TRUE),2))
head(sessions_tidy)

hashedEmail,avg_session_length
<chr>,<dbl>
0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc93355ccc95b45423367832,53.00
060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe8e1cf0eee9a7b67967,30.00
0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce0290fa0437ce0b97f387,11.00
0d4d71be33e2bc7266ee4983002bd930f69d304288a8663529c875f40f1750f3,32.15
0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab9c1ff1a0e7ca200b3a,35.00
11006065e9412650e99eea4a4aaaf0399bc338006f85e80cc82d18b49f0e2aa4,10.00


In [4]:
# Take out Age ,subscribe and hashedEmail from the player dataset to clean up players.csv
players_tidy <- players_origin |>
                    select(Age,subscribe,hashedEmail)  
head(players_tidy)

Age,subscribe,hashedEmail
<dbl>,<lgl>,<chr>
9,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d
17,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9
17,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28
21,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5
21,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e
17,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977


In [8]:
# Merge two datasets to make the final clean data
tidy_data <- players_tidy |>
                left_join(sessions_tidy, by = "hashedEmail") |>
                drop_na(avg_session_length)
head(tidy_data)

Age,subscribe,hashedEmail,avg_session_length
<dbl>,<lgl>,<chr>,<dbl>
9,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,74.78
17,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,85.00
17,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,5.00
21,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,50.00
21,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,9.00
21,FALSE,1d2371d8a35c8831034b25bda8764539ab7db0f63938696917c447128a2540dd,5.00


In [13]:
# summary of dataset
# The summary of mean value, min value and max value of avg_session_length
tidy_data |> summarize(mean_avg_session_length = round(mean(avg_session_length, na.rm = TRUE),2),
                       min_avg_session_length = round(min(avg_session_length, na.rm = TRUE),2),
                       max_avg_session_length = round(max(avg_session_length, na.rm = TRUE),2))
# The summary of mean value, min value and max value of Age
tidy_data |> summarize(mean_age = round(mean(Age, na.rm = TRUE),2),
                            min_age = round(min(Age, na.rm = TRUE),2),
                            max_age = round(max(Age, na.rm = TRUE),2))
# The summary of players whose subscribe is FALSE
tidy_data |> filter(subscribe == FALSE) |> 
                summarize(subscribe_FALSE = n())
# The summary of players whose subscribe is TRUE
tidy_data |> filter(subscribe == TRUE) |> 
                summarize(subscribe_TRUE = n())
# table of mean value summary
tidy_data |> summarize(mean_avg_session_length = round(mean(avg_session_length, na.rm = TRUE),2),
                                        mean_age = round(mean(Age, na.rm = TRUE),2)) |>
                              pivot_longer(cols = everything(),
                                           names_to = "variable", 
                                           values_to = "mean_value")

mean_avg_session_length,min_avg_session_length,max_avg_session_length
<dbl>,<dbl>,<dbl>
32.51,5,150.34


mean_age,min_age,max_age
<dbl>,<dbl>,<dbl>
21.59,9,58


subscribe_FALSE
<int>
32


subscribe_TRUE
<int>
93


variable,mean_value
<chr>,<dbl>
mean_avg_session_length,32.51
mean_age,21.59


In [15]:
set.seed(3456) 
# Randomly take 75% of the data for the training set, and 25% for testing set
data_split <- initial_split(tidy_data, prop = 0.75, strata = subscribe)  
train_data <- training(data_split)   
test_data <- testing(data_split)
# Scaling the variable and create recipe
knn_recipe <- recipe(subscribe ~ Age + avg_session_length, data = train_data) |>
            step_scale(all_predictors()) |>
            step_center(all_predictors())